## Forecasting Interest Rates using Machine Learning Techniques
### Steve Hall
### December 2019

In [1]:
# import libraries
import pandas as pd 
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
# load database of market prices into pandas
raw = pd.read_csv('./data/db_MarketPredictors.csv', sep=',')

In [3]:
print(raw.shape)
raw.head()

(7807, 197)


,Day,Date,S&P 500,S&P 600,S&P 400,Nasdaq 100,Comm. Services,Cons. Disc,Cons. Staples,Energy,...,NG6 Comdty,XB6 Comdty,GC6 Comdty,SI6 Comdty,KC6 Comdty,SB6 Comdty,C 6 Comdty,W 6 Comdty,CT6 Comdty,LH6 Comdty
0,3,12/3/2019,3113.87,981.57,1995.86,8309.260,176.69,953.4808,635.73,431.49,...,2.246,173.95,1488.8,17.460,129.80,13.60,391.50,556.00,67.82,84.775
1,2,12/2/2019,3113.87,981.57,1995.86,8309.260,176.69,953.4808,635.73,431.49,...,2.217,175.20,1488.8,17.368,129.65,13.58,391.25,554.25,67.96,84.775
2,6,11/29/2019,3140.98,993.51,2010.15,8403.685,178.24,960.7933,634.09,431.37,...,2.193,175.44,1492.5,17.487,126.70,13.62,391.00,556.50,67.39,86.150
3,5,11/28/2019,3153.63,999.99,2030.78,8444.709,178.89,968.2137,635.00,435.78,...,2.312,182.76,1480.3,17.426,126.45,13.54,389.00,548.00,68.19,86.050
4,4,11/27/2019,3153.63,999.99,2030.78,8444.709,178.89,968.2137,635.00,435.78,...,2.312,182.76,1480.3,17.426,126.45,13.54,389.00,548.00,68.19,86.050


I have downloaded 195 series of market price data from Bloomberg. The futures data is adjusted for the roll, applying the difference between the prompt and next contract 5 days prior to expiration.
<br>
<br>
*Data source: Bloomberg; daily from 1/1/1990 through 12/3/2019.* 

In [4]:
raw['Date'] =pd.to_datetime(raw.Date)
raw = raw.sort_values('Date')

In [5]:
# let's add year and month-year to the dataframe
raw['Year'] = pd.DatetimeIndex(raw['Date']).year
raw['Year_MM'] = pd.to_datetime(raw['Date']).dt.to_period('M')
raw.head()

,Day,Date,S&P 500,S&P 600,S&P 400,Nasdaq 100,Comm. Services,Cons. Disc,Cons. Staples,Energy,...,GC6 Comdty,SI6 Comdty,KC6 Comdty,SB6 Comdty,C 6 Comdty,W 6 Comdty,CT6 Comdty,LH6 Comdty,Year,Year_MM
7806,2,1990-01-01,353.40,NaN,NaN,223.835,99.89,71.2032,62.33,91.62,...,896.3,12.258,294.10,9.15,457.9375,1087.375,81.58,-22.675,1990,1990-01
7805,3,1990-01-02,359.69,NaN,NaN,227.725,101.27,73.3261,62.86,92.03,...,892.6,12.238,293.35,9.10,455.4375,1086.875,81.08,-22.875,1990,1990-01
7804,4,1990-01-03,358.76,NaN,NaN,226.100,100.47,73.1502,62.72,90.74,...,887.0,12.209,294.60,9.60,457.4375,1086.875,81.83,-22.675,1990,1990-01
7803,5,1990-01-04,355.67,NaN,NaN,224.715,97.78,72.8757,61.89,89.68,...,891.9,12.334,295.10,9.81,458.9375,1086.875,80.78,-21.100,1990,1990-01
7802,6,1990-01-05,352.20,NaN,NaN,223.140,95.64,72.3403,61.38,88.79,...,899.6,12.384,293.88,9.91,458.1875,1087.875,80.70,-20.125,1990,1990-01


In [6]:
# remove day and date from the dataframe
raw_prices = raw.iloc[:,2:-2]
raw_prices.head()

,S&P 500,S&P 600,S&P 400,Nasdaq 100,Comm. Services,Cons. Disc,Cons. Staples,Energy,Financials,Health Care,...,NG6 Comdty,XB6 Comdty,GC6 Comdty,SI6 Comdty,KC6 Comdty,SB6 Comdty,C 6 Comdty,W 6 Comdty,CT6 Comdty,LH6 Comdty
7806,353.40,NaN,NaN,223.835,99.89,71.2032,62.33,91.62,76.14,72.56,...,NaN,NaN,896.3,12.258,294.10,9.15,457.9375,1087.375,81.58,-22.675
7805,359.69,NaN,NaN,227.725,101.27,73.3261,62.86,92.03,77.35,73.63,...,NaN,NaN,892.6,12.238,293.35,9.10,455.4375,1086.875,81.08,-22.875
7804,358.76,NaN,NaN,226.100,100.47,73.1502,62.72,90.74,77.48,74.18,...,NaN,NaN,887.0,12.209,294.60,9.60,457.4375,1086.875,81.83,-22.675
7803,355.67,NaN,NaN,224.715,97.78,72.8757,61.89,89.68,76.97,73.68,...,NaN,NaN,891.9,12.334,295.10,9.81,458.9375,1086.875,80.78,-21.100
7802,352.20,NaN,NaN,223.140,95.64,72.3403,61.38,88.79,76.24,72.60,...,NaN,NaN,899.6,12.384,293.88,9.91,458.1875,1087.875,80.70,-20.125


In [7]:
# create a df of statistics of the market prices
stats_prices = raw_prices.describe().transpose()

In [8]:
# now let's calculate daily price returns for each series
raw_rets = raw_prices.pct_change()
# add back the unique months-years
raw_rets['Year_MM'] = raw['Year_MM'] 
raw_rets.head()

,S&P 500,S&P 600,S&P 400,Nasdaq 100,Comm. Services,Cons. Disc,Cons. Staples,Energy,Financials,Health Care,...,XB6 Comdty,GC6 Comdty,SI6 Comdty,KC6 Comdty,SB6 Comdty,C 6 Comdty,W 6 Comdty,CT6 Comdty,LH6 Comdty,Year_MM
7806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990-01
7805,0.017799,NaN,NaN,0.017379,0.013815,0.029815,0.008503,0.004475,0.015892,0.014746,...,NaN,-0.004128,-0.001632,-0.002550,-0.005464,-0.005459,-0.00046,-0.006129,0.008820,1990-01
7804,-0.002586,NaN,NaN,-0.007136,-0.007900,-0.002399,-0.002227,-0.014017,0.001681,0.007470,...,NaN,-0.006274,-0.002370,0.004261,0.054945,0.004391,0.00000,0.009250,-0.008743,1990-01
7803,-0.008613,NaN,NaN,-0.006126,-0.026774,-0.003753,-0.013233,-0.011682,-0.006582,-0.006740,...,NaN,0.005524,0.010238,0.001697,0.021875,0.003279,0.00000,-0.012831,-0.069460,1990-01
7802,-0.009756,NaN,NaN,-0.007009,-0.021886,-0.007347,-0.008240,-0.009924,-0.009484,-0.014658,...,NaN,0.008633,0.004054,-0.004134,0.010194,-0.001634,0.00092,-0.000990,-0.046209,1990-01


In [9]:
# create a df of statistics of the market prices
stats_rets = raw_rets.describe().transpose()

In [10]:
# average monthly returns
monthly_rets = raw_rets.groupby(['Year_MM']).sum()
monthly_rets.head()

,S&P 500,S&P 600,S&P 400,Nasdaq 100,Comm. Services,Cons. Disc,Cons. Staples,Energy,Financials,Health Care,...,NG6 Comdty,XB6 Comdty,GC6 Comdty,SI6 Comdty,KC6 Comdty,SB6 Comdty,C 6 Comdty,W 6 Comdty,CT6 Comdty,LH6 Comdty
Year_MM,,,,,,,,,,,,,,,,,,,,,
1990-01,-0.069845,0.0,0.0,-0.100755,-0.135176,-0.067095,-0.099511,-0.036108,-0.110280,-0.066651,...,0.000000,0.0,0.011318,-0.000954,-0.009802,0.109309,-0.008090,-0.008758,0.015696,-0.127050
1990-02,0.009007,0.0,0.0,0.029817,-0.004466,0.018933,-0.015465,0.018433,0.032155,-0.027902,...,0.000000,0.0,-0.010069,-0.009129,0.040157,-0.023497,0.019195,-0.000448,-0.008390,-0.019234
1990-03,0.024480,0.0,0.0,0.025821,0.049772,0.029731,0.056362,0.003447,-0.031671,0.033875,...,0.000000,0.0,-0.047333,-0.019215,-0.004865,0.089666,0.003372,-0.016800,0.006884,-0.078533
1990-04,-0.026771,0.0,0.0,-0.034203,-0.036512,-0.037562,0.012241,-0.038407,-0.044783,0.003263,...,-0.003993,0.0,-0.002698,-0.002137,-0.001632,0.044573,0.035624,-0.001363,0.019265,-0.225010
1990-05,0.088675,0.0,0.0,0.138619,0.083135,0.102705,0.098296,0.060451,0.115542,0.121566,...,0.004011,0.0,-0.010081,0.006470,0.005874,-0.174647,-0.004787,0.005805,0.028105,0.032635


In [11]:
# number of months
len(monthly_rets)

360

In [12]:
# instead of summing daily returns we could calculate rolling 21 day returns (roughly 21 days/ month)
# let's create a function
def rolling_pct_rets(df, periods):
    rets = (df.shift(periods=periods, fill_value=0)/ df) - 1
    rets = rets[:periods]
    return rets

In [13]:
monthly_rets2 = rolling_pct_rets(raw_prices, -21)

In [14]:
# Now let's compare both approaches to calculating average monthly returns
stats_Mrets = monthly_rets.describe().transpose()
stats_Mrets2 = monthly_rets2.describe().transpose()

In [15]:
ave_ret_comp = pd.concat([stats_Mrets['mean']*12, stats_Mrets2['mean']*12], axis=1) #multiply by 12 to annualize
ave_ret_comp.head(15)

,mean,mean
S&P 500,0.087727,0.082678
S&P 600,0.094700,0.103577
S&P 400,0.117110,0.116087
Nasdaq 100,0.155816,0.146613
Comm. Services,0.040427,0.037204
Cons. Disc,0.106403,0.101823
Cons. Staples,0.088267,0.085350
Energy,0.078739,0.070067
Financials,0.098550,0.087583
Health Care,0.108648,0.102397


You can see the average annualized monthly returns are fairly similar across indices and sectors.

## Data Preprocessing

Let's start by selecting monthly changes in interest rates. We will use the 10-year Treasury yield as our proxy. 

In [17]:
# Our label or response variable
TenYr = monthly_rets2['USGG10YR Index']
TenYr.head(22)

7806    0.071330
7805    0.061538
7804    0.055054
7803    0.065479
7802    0.067009
7801    0.070894
7800    0.065801
7799    0.051784
7798    0.030582
7797    0.035697
7796    0.025515
7795    0.021596
7794    0.027103
7793    0.013248
7792    0.029883
7791    0.043410
7790    0.030423
7789    0.020679
7788    0.014739
7787   -0.000591
7786   -0.014177
7785    0.001647
Name: USGG10YR Index, dtype: float64

In [26]:
# Now we need to shift it forward 21 days since we want to predict forward changes in rates
Fwd_TenYr = TenYr.shift(periods=-21)
print(len(Fwd_TenYr))
Fwd_TenYr.head()
# note that the 22nd observation is now the first obervation (0.001647)

7786


7806    0.001647
7805    0.019363
7804    0.015809
7803    0.016129
7802    0.005048
Name: USGG10YR Index, dtype: float64

In [25]:
# Get rid of zeros
# Fwd_TenYr = Fwd_TenYr.loc[(Fwd_TenYr!=0)]
# print(len(Fwd_TenYr))
# Fwd_TenYr.head()

In [34]:
# Let's select a sample of the data that we want to use for model development
# I'm going to use the last 5000 days (~20 years of data)
n = 5000
select_Y = Fwd_TenYr.iloc[-n:]
print(select_Y.shape)

(5000,)


In [31]:
# Create a list of features or predictors
features = monthly_rets2
print(features.shape)
features.head()

(7786, 195)


,S&P 500,S&P 600,S&P 400,Nasdaq 100,Comm. Services,Cons. Disc,Cons. Staples,Energy,Financials,Health Care,...,NG6 Comdty,XB6 Comdty,GC6 Comdty,SI6 Comdty,KC6 Comdty,SB6 Comdty,C 6 Comdty,W 6 Comdty,CT6 Comdty,LH6 Comdty
7806,-0.086078,NaN,NaN,-0.116961,-0.151467,-0.086740,-0.117600,-0.052281,-0.125558,-0.078418,...,NaN,NaN,0.018297,-0.000163,-0.008841,0.093989,-0.004367,-0.008277,0.007968,-0.130099
7805,-0.085101,NaN,NaN,-0.113229,-0.141108,-0.093691,-0.103882,-0.041182,-0.119845,-0.079451,...,NaN,NaN,0.015348,0.000490,-0.007329,0.116484,-0.002745,-0.008281,0.021584,-0.131148
7804,-0.083538,NaN,NaN,-0.101968,-0.135563,-0.094525,-0.102041,-0.027661,-0.122354,-0.081289,...,NaN,NaN,0.028410,0.011467,-0.009335,0.070833,-0.008198,-0.008281,0.009165,-0.116869
7803,-0.069587,NaN,NaN,-0.084997,-0.104214,-0.083627,-0.083535,-0.008809,-0.113811,-0.065147,...,NaN,NaN,0.022873,0.002027,-0.011420,0.051988,-0.016342,-0.006901,0.025130,-0.029621
7802,-0.057780,NaN,NaN,-0.074617,-0.078524,-0.074156,-0.074943,0.003604,-0.101522,-0.052617,...,NaN,NaN,0.019008,-0.000081,-0.008677,0.048436,-0.016914,-0.009192,0.028253,0.012422


In [32]:
stats_features = features.describe().transpose()
# I want to use features with at least 5000 days (20 years)
select_X = features.iloc[-n:,:]
select_X = select_X.dropna(axis='columns')
print(select_X.shape)
select_X.tail()

(5000, 181)


,S&P 500,S&P 600,S&P 400,Nasdaq 100,Comm. Services,Cons. Disc,Cons. Staples,Energy,Financials,Health Care,...,CL6 Comdty,NG6 Comdty,GC6 Comdty,SI6 Comdty,KC6 Comdty,SB6 Comdty,C 6 Comdty,W 6 Comdty,CT6 Comdty,LH6 Comdty
25,0.038441,0.020023,0.028118,0.049357,0.045957,0.020577,0.014880,-0.005000,0.043229,0.058950,...,0.042608,-0.010274,-0.023871,-0.048176,0.136119,0.002963,-0.042462,0.002286,0.004419,-0.050745
24,0.035073,0.024674,0.030884,0.044735,0.043943,0.014460,0.010181,0.016562,0.044733,0.050774,...,0.060923,-0.024473,-0.027718,-0.050095,0.136119,0.002963,-0.043639,0.008744,-0.003507,-0.056211
23,0.034047,0.029171,0.028015,0.039567,0.037365,0.011356,0.010840,0.011039,0.048257,0.048477,...,0.023265,-0.072727,-0.031096,-0.057812,0.113357,0.002208,-0.036352,0.029126,-0.010135,-0.062058
22,0.015312,0.001173,0.006231,0.018146,0.019856,-0.002813,0.013439,-0.013421,0.026887,0.042117,...,0.001461,-0.071608,-0.031549,-0.064325,0.118637,-0.003668,-0.035736,0.010483,-0.001616,-0.078032
21,0.011565,-0.007322,0.000381,0.012008,0.015752,-0.003947,0.023011,-0.043514,0.017821,0.045729,...,-0.014179,-0.076100,-0.031549,-0.060533,0.119931,-0.002933,-0.030341,0.022999,-0.004404,-0.070195


In [39]:
# let's merge select_Y and select_X to ensure they are the same size and remove any NA
select_dataset = pd.concat((select_Y, select_X), axis=1)
select_dataset.head()

,USGG10YR Index,S&P 500,S&P 600,S&P 400,Nasdaq 100,Comm. Services,Cons. Disc,Cons. Staples,Energy,Financials,...,CL6 Comdty,NG6 Comdty,GC6 Comdty,SI6 Comdty,KC6 Comdty,SB6 Comdty,C 6 Comdty,W 6 Comdty,CT6 Comdty,LH6 Comdty
5020,-0.025144,-0.048279,-0.040499,-0.033593,-0.133852,0.001068,-0.025910,0.039623,0.014705,0.026517,...,0.010773,0.029122,-0.011198,-0.009051,0.006461,-0.015968,0.016079,-0.001259,-0.034243,0.009960
5019,-0.004272,-0.037507,-0.047494,-0.033203,-0.107697,0.027943,-0.035745,0.070799,-0.005216,0.028953,...,-0.023525,0.014400,-0.007827,-0.006043,-0.002454,-0.035644,0.023720,0.004432,-0.025785,0.005010
5018,0.007398,-0.062243,-0.082400,-0.064734,-0.162247,0.013234,-0.053670,0.069070,-0.007613,-0.013836,...,-0.032454,0.008143,-0.009871,-0.003034,-0.007306,-0.040462,0.023750,0.001892,-0.021849,0.006944
5017,0.007385,-0.061874,-0.073195,-0.056912,-0.129683,-0.012094,-0.066365,0.056126,0.021933,-0.044554,...,0.039267,0.019032,-0.006819,-0.001735,-0.004129,0.027668,0.015557,0.002528,-0.022385,0.017857
5016,0.014380,-0.068652,-0.082180,-0.076917,-0.139867,-0.027295,-0.076512,0.067455,0.020499,-0.087815,...,0.023247,0.009356,-0.003069,0.005442,-0.003744,0.106640,0.021793,0.014562,-0.029257,-0.030452


In [40]:
before_rows = len(select_dataset)
select_dataset = select_dataset.dropna()
after_rows = len(select_dataset)
print(before_rows-after_rows)
# we dropped 21 rows which is consistent with our lookback period

21


In [43]:
# Lastly, let's select our Y and X variables for modeling
Y = select_dataset.iloc[:,0]
Y.head()

5020   -0.025144
5019   -0.004272
5018    0.007398
5017    0.007385
5016    0.014380
Name: USGG10YR Index, dtype: float64

In [45]:
X = select_dataset.iloc[:,1:]
X.head()

,S&P 500,S&P 600,S&P 400,Nasdaq 100,Comm. Services,Cons. Disc,Cons. Staples,Energy,Financials,Health Care,...,CL6 Comdty,NG6 Comdty,GC6 Comdty,SI6 Comdty,KC6 Comdty,SB6 Comdty,C 6 Comdty,W 6 Comdty,CT6 Comdty,LH6 Comdty
5020,-0.048279,-0.040499,-0.033593,-0.133852,0.001068,-0.025910,0.039623,0.014705,0.026517,0.048557,...,0.010773,0.029122,-0.011198,-0.009051,0.006461,-0.015968,0.016079,-0.001259,-0.034243,0.009960
5019,-0.037507,-0.047494,-0.033203,-0.107697,0.027943,-0.035745,0.070799,-0.005216,0.028953,0.100598,...,-0.023525,0.014400,-0.007827,-0.006043,-0.002454,-0.035644,0.023720,0.004432,-0.025785,0.005010
5018,-0.062243,-0.082400,-0.064734,-0.162247,0.013234,-0.053670,0.069070,-0.007613,-0.013836,0.076462,...,-0.032454,0.008143,-0.009871,-0.003034,-0.007306,-0.040462,0.023750,0.001892,-0.021849,0.006944
5017,-0.061874,-0.073195,-0.056912,-0.129683,-0.012094,-0.066365,0.056126,0.021933,-0.044554,0.060665,...,0.039267,0.019032,-0.006819,-0.001735,-0.004129,0.027668,0.015557,0.002528,-0.022385,0.017857
5016,-0.068652,-0.082180,-0.076917,-0.139867,-0.027295,-0.076512,0.067455,0.020499,-0.087815,0.079171,...,0.023247,0.009356,-0.003069,0.005442,-0.003744,0.106640,0.021793,0.014562,-0.029257,-0.030452
